<a href="https://colab.research.google.com/github/wolffg777/ait_datascience/blob/main/MusicRec01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

TODO:
WORD2VEC KERAS CLASSIFICATION PROBLEM
- first clean up
- testing it to see improvement
- implementing primary_genres and secondary_genres as inputs
- implement output as album-artist name pair
- implementing dates (as continuous)
- easier things first
- spotify data integration


-brainstorm an idea for the name of the independent study on askbanner (limited to 30 characters)




check - train on domain text
notcheck - pretrained w2v
notcheck - use pretrained with domain adaptation  



In [1]:
#Downloading Google's Word2Vec model
# import gensim.downloader as api
# path = api.load("word2vec-google-news-300", return_path=True)
# print(path)

#Mounting Google Drive
from google.colab import drive
drive.mount('/content/drive')

#Downloading Google's Word2Vec model to Google Drive
# datafiles_folder_path = "/content/drive/MyDrive/DataFiles/word2vec-google-news-300"
# !cp $path $datafiles_folder_path

#installs
from google.colab import files
files.upload()
!pip install kaggle
!chmod 600 /root/.kaggle/kaggle.json
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!kaggle datasets download -d tobennao/rym-top-5000
!unzip rym-top-5000.zip
!pip install spotipy
!pip install -U -q PyDrive

# #Decompress w2v model
# import gzip
# with gzip.open('/content/drive/MyDrive/DataFiles/word2vec-google-news-300', 'rb') as f_in:
#     with open('/content/drive/MyDrive/DataFiles/word2vec-google-news-300.bin', 'wb') as f_out:
#         f_out.write(f_in.read())

Mounted at /content/drive


Saving kaggle.json to kaggle.json
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
100% 863k/863k [00:00<00:00, 925kB/s]
100% 863k/863k [00:00<00:00, 925kB/s]
Archive:  rym-top-5000.zip
  inflating: rym_clean1.csv          
  inflating: rym_raw1.csv            
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.3/250.3 kB 4.6 MB/s eta 0:00:00


In [2]:
#imports
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
import warnings
warnings.filterwarnings('ignore')
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from gensim.models import KeyedVectors
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.cluster import KMeans

In [3]:
# Load w2v model
model = KeyedVectors.load_word2vec_format('/content/drive/MyDrive/DataFiles/word2vec-google-news-300.bin', binary=True)

In [4]:
def loadData():
  #Load data
  df = pd.read_csv('rym_clean1.csv')

  # Remove unnecessary columns
  df = df.drop(columns=['avg_rating'])
  df = df.drop(columns=['rating_count'])
  df = df.drop(columns=['review_count'])
  df = df.drop(columns=['Unnamed: 0'])
  df = df.drop(columns=['position'])
  df = df.drop(columns=['release_type'])
  df = df.drop(columns=['artist_name'])
  df = df.drop(columns=['release_date'])

  return df

In [5]:
def filter(df):
  #Reformat descriptors properly
  df['descriptors'] = df['descriptors'].str.split(', ')

  # Removing rare descriptors
  MIN_OCCURRENCE_OF_DESCRIPTORS = 5

  # Flatten the list of all descriptors and count their occurrences
  all_descriptors = [descriptor for descriptors in df['descriptors'] for descriptor in descriptors]
  descriptor_frequency = pd.Series(all_descriptors).value_counts()

  # Filter out rare descriptors
  rare_descriptors = descriptor_frequency[descriptor_frequency < MIN_OCCURRENCE_OF_DESCRIPTORS].index
  df['descriptors'] = df['descriptors'].apply(lambda descriptors: [descriptor for descriptor in descriptors if descriptor not in rare_descriptors])

  # Removing albums below a minimum number of descriptors
  MIN_DESCRIPTORS_PER_ALBUM = 5  # Minimum number of descriptors per album
  df = df[df['descriptors'].map(len) >= MIN_DESCRIPTORS_PER_ALBUM]

  return df


In [6]:
def tokenize(df):
  #Tokenize descriptors
  texts = df['descriptors'].tolist()
  tokenizer = Tokenizer()
  tokenizer.fit_on_texts(texts)
  sequences = tokenizer.texts_to_sequences(texts)
  return sequences

In [7]:
def vectorize(df):
  def descriptors_to_vector(descriptors):
      #Vectorize descriptors

      vectors = [model[word] for word in descriptors if word in model]
      if vectors:
          return np.mean(vectors, axis=0)
      else:
          return np.zeros(model.vector_size)

  # Apply the function to the dataframe
  df['vector'] = df['descriptors'].apply(descriptors_to_vector)

  # Convert df to nparray
  df_nparray = np.array(df['vector'].tolist())
  return df


In [8]:
def makeTokenizer(df):
  texts = df['descriptors'].tolist()
  tokenizer = Tokenizer()
  tokenizer.fit_on_texts(texts)
  return tokenizer

In [9]:
def embed(df):
  tokenizer = makeTokenizer(df)
  embedding_dim = 300  # As the Word2Vec model you downloaded has 300 dimensions
  vocab_size = len(tokenizer.word_index) + 1

  # Create an embedding matrix
  embedding_matrix = np.zeros((vocab_size, embedding_dim))
  for word, i in tokenizer.word_index.items():  # tokenizer should be the tokenizer you used to process your text
      if word in model:
          embedding_vector = model[word]
          if embedding_vector is not None:
              embedding_matrix[i] = embedding_vector
  return embedding_matrix



In [10]:
def encode(df):
  encoder = LabelEncoder()
  df['release_name_label'] = encoder.fit_transform(df['release_name'])
  return df

In [ ]:
# Model 1

def model1(X_train, X_val, y_train, y_val, df, df_nparray, embedding_matrix) :

  modelSeq = Sequential([
      Embedding(input_dim=len(makeTokenizer(df).word_index) + 1, output_dim=300, input_length=df_nparray.shape[1],
               weights=[embedding_matrix], trainable=False),  # Using pre-trained embeddings and setting them non-trainable

     #change lstm to deep feedforward nn
      LSTM(128, return_sequences=True),  # Return sequences if you want to stack LSTMs
     Dropout(0.2),  # Add dropout for regularization
      LSTM(64),  # Another LSTM layer
      Dropout(0.2),
      Dense(256, activation='relu'),
      Dropout(0.2),
      Dense(128, activation='relu'),
      Dropout(0.2),
      Dense(64, activation='relu'),
      Dropout(0.2),
      Dense(len(df['release_name'].unique()), activation='softmax')
  ])

  modelSeq.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

  # Callbacks
  early_stop = EarlyStopping(monitor='val_loss', patience=3, verbose=1, restore_best_weights=True)
  model_checkpoint = ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True, verbose=1)

  # Assuming you've created a validation set called X_val and y_val
  history = modelSeq.fit(X, y, epochs=25, batch_size=64, validation_data=(X_val, y_val), callbacks=[early_stop, model_checkpoint])


In [11]:
# Model 2
# does not use embedding, simpler design, sequential not an lstm

def model2(X_train, X_val, y_train, y_val, df, df_nparray):
  modelSeq = Sequential([
     Dense(128, activation='relu', input_shape=(df_nparray.shape[1],)),
      Dropout(0.2),
      Dense(64, activation='relu'),
      Dropout(0.2),
      Dense(32, activation='relu'),
      Dropout(0.2),
      Dense(len(df['release_name'].unique()), activation='softmax')
    ])

  modelSeq.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

  # Callbacks remain the same
  early_stop = EarlyStopping(monitor='val_loss', patience=3, verbose=1, restore_best_weights=True)
  model_checkpoint = ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True, verbose=1)

  # Train the model
  history = modelSeq.fit(X_train, y_train, epochs=100, batch_size=64, validation_data=(X_val, y_val), callbacks=[early_stop, model_checkpoint])


In [ ]:
# Method1: Baseline Data (descriptors --> album titles, really bad)
df = loadData()
df = filter(df)
sequences = tokenize(df)
df = vectorize(df)
df = encode(df)

df_nparray = np.array(df['vector'].tolist())

# Prepare training data
X = np.array(df_nparray, dtype='float32')
y = df['release_name_label'].values

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [12]:
# Method2: Clustering Albums
df = loadData()
df = filter(df)
sequences = tokenize(df)
df = vectorize(df)
df = encode(df)

df_nparray = np.array(df['vector'].tolist())

In [ ]:
# import matplotlib.pyplot as plt
# inertia = []
# for k in range(1, 1000, 50):  # Adjust the range of k as needed
#     kmeans = KMeans(n_clusters=k, random_state=42)
#     kmeans.fit_predict(df_nparray)  # Assuming df_nparray is your feature array
#     inertia.append(kmeans.inertia_)
# plt.figure(figsize=(12, 6))
# plt.plot(range(1, 1000, 50), inertia, marker='o')
# plt.title('Elbow Method')
# plt.xlabel('Number of clusters')
# plt.ylabel('Inertia')
# plt.show()

In [13]:
k = 100
kmeans = KMeans(n_clusters=k, random_state=42)
df['cluster_label'] = kmeans.fit_predict(df_nparray)

X = np.array(df_nparray, dtype='float32')
y = df['cluster_label'].values

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

k = 50 --> ~70% accuracy,
k = 100 --> 55% accuracy,
k = 250 --> 35% accuracy,
k = 500 --> ~30% accuracy,


In [ ]:
cluster_number = 0  # Replace with the cluster number you are interested in

# Filter the DataFrame to only include rows belonging to the specified cluster
cluster_members = df[df['cluster_label'] == cluster_number]

# Print the information of interest from these rows
# For example, printing album names and descriptors
print(cluster_members[['release_name', 'descriptors']])

print(len(cluster_members[['release_name', 'descriptors']]))

                                       release_name  \
66                                     Led Zeppelin   
84                                  Led Zeppelin II   
141   Whatever People Say I Am, That's What I'm Not   
216                                    Aladdin Sane   
254                                  Sticky Fingers   
...                                             ...   
4893                                          Loose   
4924                                  A Bigger Bang   
4945                                Scissor Sisters   
4952                               Feed the Animals   
4972                                       Elephunk   

                                            descriptors  
66    [energetic, heavy, malevocals, raw, sexual, pa...  
84    [energetic, sexual, heavy, malevocals, raw, he...  
141   [urban, energetic, sarcastic, malevocals, raw,...  
216   [malevocals, melodic, playful, energetic, pass...  
254   [drugs, sexual, malevocals, energetic, hedo

In [ ]:
# model1 = None
# model1 = model1(X_train, X_val, y_train, y_val, df, df_nparray, embed(df))

In [14]:
model2 = model2(X_train, X_val, y_train, y_val, df, df_nparray)

Epoch 1/100
59/62 [===========================>..] - ETA: 0s - loss: 6.4973 - accuracy: 0.0101
Epoch 1: val_loss improved from inf to 4.65726, saving model to best_model.h5
62/62 [==============================] - 10s 11ms/step - loss: 6.4215 - accuracy: 0.0104 - val_loss: 4.6573 - val_accuracy: 0.0121
Epoch 2/100
54/62 [=========================>....] - ETA: 0s - loss: 4.6776 - accuracy: 0.0150
Epoch 2: val_loss improved from 4.65726 to 4.58693, saving model to best_model.h5
62/62 [==============================] - 0s 8ms/step - loss: 4.6661 - accuracy: 0.0144 - val_loss: 4.5869 - val_accuracy: 0.0182
Epoch 3/100
62/62 [==============================] - ETA: 0s - loss: 4.5632 - accuracy: 0.0230
Epoch 3: val_loss improved from 4.58693 to 4.37435, saving model to best_model.h5
62/62 [==============================] - 0s 8ms/step - loss: 4.5632 - accuracy: 0.0230 - val_loss: 4.3744 - val_accuracy: 0.0354
Epoch 4/100
62/62 [==============================] - ETA: 0s - loss: 4.1792 - accura

1. get lyrics (implement them)
2. maybe take out some of the descriptors in the train/test split (20% of the descriptors are not used in training, in order to test if users use words not in vocab what would happen)
3. get some F1 scores
4. get reviews (discog), then implement those
5. 1 IMPLEMENT GENRES 1